In [1]:
import networkx as nx
import numpy as np
import itertools
from scipy.sparse import csr_matrix

# 创建一个无向图
G = nx.Graph()
G.add_nodes_from([1, 2, 3, 4, 5])
G.add_edges_from([(1, 2), (2, 3), (3, 4), (4, 5)])

# 构建邻接矩阵
A = nx.adjacency_matrix(G)
n = A.shape[0]

# 将邻接矩阵转换为稀疏矩阵
A = csr_matrix(A)

# 定义目标函数
def objective_function(x):
    return x.dot(A.dot(1 - x))

# 枚举所有可能的割集
max_cut_value = 0
max_cut = None
for i in range(1, n // 2 + 1):
    for subset in itertools.combinations(range(n), i):
        x = np.zeros(n)
        x[list(subset)] = 1
        cut_value = objective_function(x)
        if cut_value > max_cut_value:
            max_cut_value = cut_value
            max_cut = subset

print("最大割集：", max_cut)
print("最大割值：", max_cut_value)

最大割集： (1, 3)
最大割值： 4.0


C:\Users\Newt\AppData\Local\Temp\ipykernel_14308\2542940655.py:12: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G)


In [4]:
# 创建无向图
graph = {
    0: [1, 2],
    1: [0, 2, 3],
    2: [0, 1, 4],
    3: [1, 4],
    4: [2, 3]
}

# 初始化残余网络
def init_residual_network(graph):
    residual_network = {}
    for node in graph:
        residual_network[node] = {}
        for neighbor in graph[node]:
            residual_network[node][neighbor] = graph[node].count(neighbor)
    return residual_network

# 寻找增广路径
def find_augmenting_path(residual_network, source, sink, path):
    if source == sink:
        return True
    for neighbor in residual_network[source]:
        if neighbor not in path and residual_network[source][neighbor] > 0:
            path.append(neighbor)
            if find_augmenting_path(residual_network, neighbor, sink, path):
                return True
            path.pop()
    return False

# Ford-Fulkerson算法
def ford_fulkerson(graph, source, sink):
    residual_network = init_residual_network(graph)
    max_flow = 0
    while find_augmenting_path(residual_network, source, sink, []):
        path_flow = float('inf')
        s = sink
        while s != source:
            u = path.pop()
            v = path.pop()
            path_flow = min(path_flow, residual_network[u][v])
            s = u
        max_flow += path_flow
        v = sink
        while v != source:
            u = path.pop()
            residual_network[u][v] -= path_flow
            residual_network[v][u] += path_flow
            v = u
    return max_flow

# 计算最大割
def max_cut(graph, source, sink):
    max_flow = ford_fulkerson(graph, source, sink)
    visited = set()
    path = []
    dfs(graph, source, sink, visited, path)
    return max_flow, [node for node in graph if node not in visited]

# DFS遍历图
def dfs(graph, node, sink, visited, path):
    visited.add(node)
    path.append(node)
    if node == sink:
        return True
    for neighbor in graph[node]:
        if neighbor not in visited and graph[node][neighbor] > 0:
            if dfs(graph, neighbor, sink, visited, path):
                return True
    path.pop()
    return False

# 测试
source = 0
sink = 4
print("最大割为：", max_cut(graph, source, sink))


NameError: name 'path' is not defined

In [5]:
 
'''
图的存储结构: 二维矩阵
编码方案: 一个个体为一个字符串,每个位置代表每个节点所属
示例:
默认顺序ABCDEFGH
str=0b00110010 意为ABEFH属于0子树,CDG属于1子树
范围128(100000000)到255(11111111)
选择方案:轮盘赌
交叉:选择一个点位
变异算子:单点变异,就是某一点突然变成另一个子树
条件:n代结束
环境:Python
开发平台:vscode
'''
import math
import random
import copy
import pdb
 
 
'''
popnum:单位种群中个体数量,cistern:存储单个种群的数据池,lbound,ubound:个体的取值范围
precision:取值的精度,n:问题的维度
'''
def initpop():
    cistern=[]
    for i in range(popnum):
        particle=[]
        for j in range(n):
            particle.append(round(random.uniform(lbound,ubound),precision))
        cistern.append([particle,[],None])
    return cistern
 
 
# def D2B(num):  #编码(十进制2二进制)
#     return bin(round((num-lbound)*(pow(2,l)-1)/(ubound-lbound)))
    
# def B2D(num):  #解码(二进制2十进制)
#     return round(lbound+(ubound-lbound)/(pow(2,l)-1)*int(num,2),2)
    
def trans(cistern): #对种群进行二进制编码
    for i in cistern:
        for j in range(n):
            i[1].append(bin(int(i[0][j])))
    return cistern
    
def fitness(cistern): #对种群进行评估
    for i in cistern:
        i[2]=fit(i[0])
    return cistern
def select(cistern):
    reservoir=[]
    choice=[]
    resualt={}
    s=0
    for i in cistern:  #将种群中所有的适应值相加
        s=s+i[2]
    for j in cistern:  #求每个个体的占比
        try:
            reservoir.append(j[2]/s)
        except:
            reservoir.append(1/popnum)
    for k in range(1,popnum):  #求每个个体的累计占比
        reservoir[k]=reservoir[k]+reservoir[k-1]
    reservoir.insert(0,0)   #补充轮盘的开端，设为0
    for c in range(popnum):  #选择次数为种群容量
        r=random.random()    #选择概率
        for t in range(1,len(reservoir)-1):
            if r<reservoir[t] and r>reservoir[t-1]:
                choice.append(t-1)
    for d in set(choice):   #采用字典的方式存储各个个体以及被选中的次数
        resualt[d]=choice.count(d)
    resualt=sorted(resualt.items(), key=lambda item: item[1],reverse=True) #以被选中的次数大小排序
    choice=[]
    for u in resualt:  
      choice.append(u[0])
    return choice #返回选择的结果(下标)
 
def crossover(a,b): #染色体(个体)单位
    ch=random.random()  #交叉概率
    if ch<overchance:
        length=9999999
        for i in a[1]+b[1]:  #对较小长度的二进制选择交叉位作为各个维度的交叉位
            if len(i)<length:
                length=len(i)
        r=random.randint(2,length-1) #选择交叉位
        temp1,temp2=list(a[1][0]),list(b[1][0])
        temp1[r],temp2[r]=temp2[r],temp1[r]
        a[1][0],b[1][0]=''.join(temp1),''.join(temp2)
        # for j in range(n):  #对二进制数值进行剪切和拼接
        #     temp=a[1][j][r:]
        #     a[1][j]=a[1][j][:r]+b[1][j][r:]
        #     b[1][j]=b[1][j][:r]+temp
        #     a[0][j]=int(a[1][j],2)  #更改十进制数值
        #     b[0][j]=int(b[1][j],2)
        a[2]=fit(a[0])    #重新评估函数
        b[2]=fit(b[0])
 
def variation(cistern):
    for i in cistern:
        c=random.random()  #变异概率
        if c<variantchance:  #达成条件进行变异
            m=99999
            for x in i[1]:  #以长度较小的二进制值选择点位
                if len(x)<m:
                    m=len(x)
            r=random.randint(2,m-1) #变异点位
            for j in range(n):    #更换点位的值
                if i[1][j][r]=='0':  
                    i[1][j]=replace_char(i[1][j],'1',r)
                    i[0][j]=int(i[1][j],2) 
                else:
                    i[1][j]=replace_char(i[1][j],'0',r)
                    i[0][j]=int(i[1][j],2)
                while i[0][0]==255:
                    i[0][0]=random.randint(lbound,ubound)
                i[2]=fit(i[0]) #重新评估函数
    return cistern
def replace_char(string,char,index): #单字符交叉，用于“变异”
    string=list(string)
    string[index]=char
    return ''.join(string)
    
# def fit(x):  #目标函数，函数值即为适应度。
#     value=round(10*math.sin(5*x[0])+7*abs(x[1]-5)+10,precision)
#     return value
    
def match(choice):  #匹配函数，防止选选择的个体个数为奇数。
    if len(choice)%2!=0:
        choice.pop()
    return choice
    
def best(cistern,Best):  #最优个体保持函数
    tem=sorted(cistern,key=lambda x:x[2],reverse=True)[0]
    if not Best:
        Best=copy.deepcopy(tem)
    elif tem[2]>Best[2]:
        Best=copy.deepcopy(tem)
    return Best
 
 
Graph=[[0,1,0,1,0,0,1,0],
        [1,0,0,0,1,1,0,0],
        [0,0,0,0,0,1,0,1],
        [1,0,0,0,0,1,0,0],
        [0,1,0,0,0,0,1,0],
        [0,1,1,1,0,0,0,0],
        [1,0,0,0,1,0,0,0],
        [0,0,1,0,0,0,0,0]]
# def DirectConnect(point): #检查直接相连的点
#     ans=set()
#     for i in range(8):
#         if Graph[point][i]:
#             ans.add(i)
#     return ans
def check(x,y): #检查一步能到达是否有两个路径
    return sum([Graph[y][i] for i in x])
def fit(x):  #目标函数，函数值即为适应度。
    simple=bin(int(x[0]))
    value=0
    left,right=set(),set()
    for i,j in enumerate(simple[2:]):
        if int(j):
            left.add(i)
        else:
            right.add(i)
    # pdb.set_trace()
    S=0
    for i in left:
        S+=check(left,i)
    if S/2<len(left)-1:
        return 0
    S=0
    for i in right:
        S+=check(right,i)
    if S/2<len(right)-1:
        return 0
    for i in left:
        for g,k in enumerate(Graph[i]):
            if int(k):
                if g in right:
                    value+=1
    return value
 
 
if __name__ == '__main__':
    # fit([164])
    popnum=50
    ubound=255
    lbound=128
    precision=0
    l=0
    while pow(2,l)<(ubound-lbound)/(1/pow(10,precision)): #计算所需二进制长度
        l=l+1
    n=1 #2-D 
    overchance=0.8
    variantchance=0.3
    Best=[]
    a=initpop()
    trans(a)
    fitness(a)
    for j in range(100000):
        s=select(a)
        if s:
            c=match(s)
            for i in range(0,len(c),2):
                crossover(a[c[i]],a[c[i+1]])
        variation(a)
        Best=best(a,Best)
        print(Best)
 

[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b11111001'], 3]
[[251.0], ['0b